In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from time import time
from copy import deepcopy
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT_PATH = Path().absolute().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from peq import PEQ
from constants import DEFAULT_FS, PEQ_CONFIGS

In [3]:
dir_path = Path().absolute()
config_names = [
    ['4_PEAKING_WITH_LOW_SHELF'],
    ['4_PEAKING_WITH_LOW_SHELF', '4_PEAKING_WITH_HIGH_SHELF'],
    ['8_PEAKING_WITH_SHELVES'],
    ['10_BAND_GRAPHIC_EQ'],
]
for names in config_names:
    dir_path.joinpath('inspection', ','.join(names)).mkdir(parents=True, exist_ok=True)

In [5]:
for names in config_names:
    for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
        fr = FrequencyResponse.read_from_csv(fp)
        peqs = fr.optimize_parametric_eq([PEQ_CONFIGS[name] for name in names], DEFAULT_FS, max_time=5)
        compound = PEQ(f=peqs[0].f, fs=DEFAULT_FS)
        for peq in peqs:
            for filt in peq.filters:
                compound.add_filter(filt)
        compound.target = peqs[0].target
        compound.sort_filters()
        fig, ax = compound.plot()
        ax.set_title(fp.name.replace('.csv', ''))
        fig.savefig(dir_path.joinpath('inspection', ','.join(names), fp.name.replace('.csv', '.png')), dpi=120)
        plt.close(fig)